# PCA

En este tutorial trabajaremos con PCA (Análisis de Componentes Principales).

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns


# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn import metrics
import multiprocessing

from scipy.stats import randint as sp_randint

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

Utilizaremos el dataset de cáncer de mama (breast cancer)

In [ ]:
data = load_breast_cancer()

In [ ]:
print(data.DESCR)

In [ ]:
# Pasamos a un Data frame
df = pd.DataFrame(data.data, columns=data.feature_names)
# Añadimos una columna con la variable objetivo
df['target'] = data.target
# Mostramos las primeras filas
df.head()

In [ ]:
df.info()

Vemos como todas las variables de entradas son `float64`. Además, no faltan valores, todas las columnas tienen 569 valores.

In [ ]:
# Número de muestras por clase
# ==============================================================================
df.target.value_counts().sort_index()

Tenemos 212 muestras que no tienen cáncer y 357 que sí tienen cáncer. Aunque están un poco desbalanceadas las clases, lo vamos a dejar así.

In [ ]:
# Pasamos de dataframe a numpy para poder trabajar con sklearn
X = df.drop('target', axis=1)
y = df['target']
# dividimos las muestras en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=42, shuffle = True)

## LogisticRegression sin PCA
Los parámetros más importantes de la implantación de sklearn (`LogisticRegression`) son:

- `penalty`: El tipo de aplicación de regularización. Sus valores pueden ser:{None, 'l2' (por defecto), 'l1', 'elascticnet'}
- `C`: (por defecto 1.0) Inverso de la fuerza de regularización; Valores más pequeños especifican una regularización más fuerte.
- `solver`: Algoritmo a utilizar en el problema de optimización. Sus valores pueden ser: {‘lbfgs’ (por defecto), ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’}. Algunas consideraciones:

Procedemos a la búsqueda de los hiperparámteros

In [ ]:
scaler = StandardScaler()
lr = LogisticRegression()

pipe_scale_lr = Pipeline([
    ('scale', scaler),
    ('lr', lr)])


param_grid = [{'lr__penalty': ['l1', 'l2', 'elascticnet'],
               'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
               'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]


inner = KFold(n_splits=3, shuffle=True, random_state=42)

#budget = 40
# Cross-validation (3-fold) para la búsqueda de hiper-parámetros
clf = GridSearchCV (estimator  = pipe_scale_lr,
                    param_grid = param_grid,
                    scoring='accuracy', #OJO!!! scoring=‘balanced_accuracy’
                    cv=inner,
                    refit=True,
                    n_jobs=-1,
                    verbose=1,
                    return_train_score=True)

np.random.seed(42)

clf.fit(X=X_train, y=y_train)

In [ ]:
resultados = pd.DataFrame(clf.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
clf.best_params_, clf.best_score_

Al poner el parámetro `refit=True` se reentrena el modelo indicando los valores óptimos en sus argumentos. Este reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en `.best_estimator_`.

In [ ]:
modelo_final = clf.best_estimator_
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

## LogisticRegression con PCA


Procedemos a la búsqueda de los hiperparámteros y añadimos PCA en el pipeline

---



In [ ]:
# El número máximo de variables de entrada es de 30

pca = PCA(n_components=30,random_state=42)

X_trans= pca.fit_transform(X)

print(f"Proporción de varianza explicada: {pca.explained_variance_ratio_}")
print(f"Proporción acumulada: {np.cumsum(pca.explained_variance_ratio_)}")


Comprobamos que con las dos primeras variables llegamos al 0.999. Con dos componentes es suficiente

In [ ]:

pca = PCA(n_components=2,random_state=42)

pipe_scale_lr = Pipeline([
    ('scale', scaler),
    ('pca', pca),
    ('lr', lr)])

param_grid = [{'lr__penalty': ['l1', 'l2', 'elascticnet'],
               'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
               'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]


inner = KFold(n_splits=3, shuffle=True, random_state=42)

#budget = 40
# Cross-validation (3-fold) para la búsqueda de hiper-parámetros
clf = GridSearchCV (estimator  = pipe_scale_lr,
                    param_grid = param_grid,
                    scoring='accuracy', #OJO!!! scoring=‘balanced_accuracy’
                    cv=inner,
                    refit=True,
                    n_jobs=-1,
                    verbose=1,
                    return_train_score=True)

np.random.seed(42)

clf.fit(X=X_train, y=y_train)

In [ ]:
resultados = pd.DataFrame(clf.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
clf.best_params_, clf.best_score_

Al poner el parámetro `refit=True` se reentrena el modelo indicando los valores óptimos en sus argumentos. Este reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en `.best_estimator_`.

In [ ]:
modelo_final = clf.best_estimator_
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

Ventaja, que lo podemos pintar porque son dos componentes principales

In [ ]:
pca=PCA(n_components=2,random_state=42)
X_trans= pca.fit_transform(X)

df_pca = pd.DataFrame(X_trans, columns=['PC1','PC2'])
df_pca['label'] = y
df_pca.head()

In [ ]:

sns.scatterplot(data=df_pca, x="PC1", y="PC2", hue='label', palette=['#1f77b4', '#ff7f0e'])

# Añadir etiquetas para los ejes y título
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Scatterplot de dos componentes principales')
# Mostrar leyenda claramente
plt.legend(title='Clase')

plt.show()

# Codificar una variable ordinal

Partimos de un DataFrame con distintos niveles de educación.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Datos con la variable categórica ordinal
data = pd.DataFrame({
    'nivel_educativo': ['Primaria', 'Secundaria', 'Bachillerato', 'Universidad', 'Maestría', 'Doctorado', 'Secundaria', 'Universidad']
})


Definimos el orden que deseamos para esos valores

In [ ]:
# Definir el orden correcto de la variable
orden_niveles = ['Primaria', 'Secundaria', 'Bachillerato', 'Universidad', 'Maestría', 'Doctorado']


Definimos el codificador con el orden establecido

In [ ]:
encoder = OrdinalEncoder(categories=[orden_niveles])

Creamos una columna más para mostrar como quedaría la codificación

In [ ]:
data['nivel_educativo_codificado'] = encoder.fit_transform(data[['nivel_educativo']])

Lo mostramos

In [ ]:
data